## Malayalam Speech Keyword generator

In [1]:
!pip install -Uqq fairseq2
!pip install  -Uqq  pydub sentencepiece
!pip install  -Uqq git+https://github.com/facebookresearch/seamless_communication.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.16.0a0 requires torch==2.1.0a0+4136153, but you have torch 2.1.1 which is incompatible.
torchtext 0.16.0a0 requires torch==2.1.0a0+4136153, but you have torch 2.1.1 which is incompatible.
torchdata 0.7.0a0 requires torch==2.1.0a0+4136153, but you have torch 2.1.1 which is incompatible.
numba 0.56.4+1.g5f1bc7084 requires numpy<1.24,>=1.18, but you have numpy 1.26.3 which is incompatible.
cupy-cuda12x 12.0.0b3 requires numpy<1.26,>=1.20, but you have numpy 1.26.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.16.0a0 requires torch==2.1.0a0+4136153, but you have torch 2.1.1 which is incompatible.
torchtext 0.16.0a0 requires torch==2.1.0a0+41

In [23]:
! pip install -Uqq git+https://github.com/huggingface/transformers.git (4.37.2)
! pip install peft

In [3]:
!pip install git+https://github.com/huggingface/transformers torch accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ltqocksm
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-ltqocksm
  Resolved https://github.com/huggingface/transformers to commit 29a2b1420633d322140062d7c76b807f41fb90aa
  distutils: /tmp/pip-build-env-rr_40ayr/normal/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-rr_40ayr/normal/local/lib/python3.10/dist-packages
  distutils: /tmp/pip-build-env-rr_40ayr/normal/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-rr_40ayr/normal/local/lib/python3.10/dist-packages
  user = False
  home = None
  root = None
  prefix = '/tmp/pip-build-env-rr_40ayr/normal'
  distutils: /tmp/pip-build-env-rr_40ayr/overlay/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-rr_40ayr/overlay/local/lib/python3.10/dist-packages
  distutils: /tmp/pip-build-env-rr_40ayr/overlay/lib/python3

In [2]:
1 + 1

2

In [3]:
import torch
import torchaudio

from seamless_communication.inference import Translator
from pydub import AudioSegment
from transformers import AutoModelForCausalLM, AutoTokenizer

import wave
import math
import os

In [2]:
model_name = "seamlessM4T_v2_large"
vocoder_name = "vocoder_v2" if model_name == "seamlessM4T_v2_large" else "vocoder_36langs"

translator = Translator(
    model_name,
    vocoder_name,
    device=torch.device("cuda:0"),
    dtype=torch.float16,
)

Using the cached checkpoint of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached checkpoint of vocoder_v2. Set `force` to `True` to download again.
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [15]:
audio_name = "audios/sample1.wav"
target_lang = "eng"

# function to calculate the duration of the input audio clip
def get_duration_wave(file_path):
   with wave.open(file_path, 'r') as audio_file:
      frame_rate = audio_file.getframerate()
      n_frames = audio_file.getnframes()
      duration = n_frames / float(frame_rate)
      return duration


duration = get_duration_wave(audio_name)
print(f"Duration: {duration:.2f} seconds")

resample_rate = 16000
t1 = 0
t2 = 20000

# Generating 'n' number of audio samples each with 20seconds duration. This is to avoid issue with the maximum sequence length
num_samples = math.ceil(duration/20)
print("number of samples ", num_samples)

# Initializing the translator model
model_name = "seamlessM4T_v2_large"
vocoder_name = "vocoder_v2" if model_name == "seamlessM4T_v2_large" else "vocoder_36langs"

translator = Translator(
    model_name,
    vocoder_name,
    device=torch.device("cuda:0"),
    dtype=torch.float16,
)

text = []
for i in range(num_samples):
    newAudio = AudioSegment.from_wav(audio_name)
    newAudio = newAudio[t1:t2]
    new_audio_name = "new_" + str(t1) + ".wav"
    newAudio.export(new_audio_name, format="wav")
    waveform, sample_rate = torchaudio.load(new_audio_name)
    resampler = torchaudio.transforms.Resample(sample_rate, resample_rate, dtype=waveform.dtype)
    resampled_waveform = resampler(waveform)
    torchaudio.save("resampled.wav", resampled_waveform, resample_rate)

    translated_text, _ = translator.predict("resampled.wav", "s2tt", target_lang)
    print(translated_text[0])
    text.append(str(translated_text[0]))
    t1 = t2
    t2 += 20000
    os.remove(new_audio_name)
os.remove("resampled.wav")

print(" ".join(text))

Duration: 192.45 seconds
number of samples  10


Using the cached checkpoint of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached checkpoint of vocoder_v2. Set `force` to `True` to download again.
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Minister P. Rajiv's reaction to the criticism of M.T. Vastevana that there is no authoritarianism in Kerala is in the K.L.F.E.D. The minister criticized that the Centre is weakening the constitutional institutions.
I don't think it's a question of whether or not we're going to be able to do it, but I think it's a question of whether or not we're going to be able to do it, and I think it's a question of whether or not we're going to be able to do it, because we're not going to be able to do it.
In India, only one state, Kerala, has a parliamentary system that is democratic.
This is the first time that such a reply or such a reference has been made by Minister P. Rajiv.
In this article, we'll look at the history, culture, and ethics of the MTA, and then we'll take a closer look at the history and culture of the MTA, and then we'll look at the history of the MTA, and then we'll take a closer look at the history and ethics of the MTA.
It's been a long time since I've been able to say that 

In [10]:
def generate_seamless_response(audio_name, target_lang = "eng"):
    duration = get_duration_wave(audio_name)
    print(f"Duration: {duration:.2f} seconds")

    resample_rate = 16000
    t1 = 0
    t2 = 20000

    # Generating 'n' number of audio samples each with 20seconds duration. This is to avoid issue with the maximum sequence length
    num_samples = math.ceil(duration/20)
    print("number of samples ", num_samples)

    # Initializing the translator model
    model_name = "seamlessM4T_v2_large"
    vocoder_name = "vocoder_v2" if model_name == "seamlessM4T_v2_large" else "vocoder_36langs"

    translator = Translator(
        model_name,
        vocoder_name,
        device=torch.device("cuda:0"),
        dtype=torch.float16,
    )

    text = []
    for i in range(num_samples):
        newAudio = AudioSegment.from_wav(audio_name)
        newAudio = newAudio[t1:t2]
        new_audio_name = "new_" + str(t1) + ".wav"
        newAudio.export(new_audio_name, format="wav")
        waveform, sample_rate = torchaudio.load(new_audio_name)
        resampler = torchaudio.transforms.Resample(sample_rate, resample_rate, dtype=waveform.dtype)
        resampled_waveform = resampler(waveform)
        torchaudio.save("resampled.wav", resampled_waveform, resample_rate)

        translated_text, _ = translator.predict("resampled.wav", "s2tt", target_lang)
        print(translated_text[0])
        text.append(str(translated_text[0]))
        t1 = t2
        t2 += 20000
        os.remove(new_audio_name)
    os.remove("resampled.wav")

    return " ".join(text)

def get_duration_wave(file_path):
   with wave.open(file_path, 'r') as audio_file:
      frame_rate = audio_file.getframerate()
      n_frames = audio_file.getnframes()
      duration = n_frames / float(frame_rate)
      return duration

In [12]:


text2 = generate_seamless_response("audios/sample2.wav")

Duration: 649.42 seconds
number of samples  33


Using the cached checkpoint of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached checkpoint of vocoder_v2. Set `force` to `True` to download again.
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


My book is coming out in about a week and people who are interested in that book will read it and they will realize that this paper Malayalam Manorama has manufactured a sentence
and put it in quotes in the headline which is not there I think a responsible publication like Malayalam Manorama when it is putting a written manuscript in quotes at least it should be true to what is written my part
Never, ever, not a single sentence in that book accuses my party, insulting my party and my work for 55 years.
It's not fair for the author to make a headline and put it in court.
From 1985 to 1985 when I was in Delhi and I was called Rita because of the emergency and it's about my experience in Delhi in the trade unions in collectively trying to build a women's organization and my
and in that the struggle of a woman to establish in that book the struggle of a young woman
to establish an independent identity in politics is part of my own experience but nowhere in this book have I ever made the ki

In [1]:
text = "Minister P. Rajiv's reaction to the criticism of M.T. Vastevana that there is no authoritarianism in Kerala is in the K.L.F.E.D. The minister criticized that the Centre is weakening the constitutional institutions. I don't think it's a question of whether or not we're going to be able to do it, but I think it's a question of whether or not we're going to be able to do it, and I think it's a question of whether or not we're going to be able to do it, because we're not going to be able to do it. In India, only one state, Kerala, has a parliamentary system that is democratic. This is the first time that such a reply or such a reference has been made by Minister P. Rajiv. In this article, we'll look at the history, culture, and ethics of the MTA, and then we'll take a closer look at the history and culture of the MTA, and then we'll look at the history of the MTA, and then we'll take a closer look at the history and ethics of the MTA. It's been a long time since I've been able to say that I'm a big supporter of the MTA, but I'm not a big supporter of the MTA, and I'm not a big supporter of the MTA, and I'm not a big supporter of the MTA, but I'm a big supporter of the MTA. It's been a long time since I've been in the media, but it's been a long time since I've been in the media, but it's been a long time since I've been in the media, and it's been a long time since I was in the media. It's not a case of a dictatorship, it's not a case of the government of Kerala, it's a case of the central government, it's a case of P.R. Rajiv's side, it's a case of the central government undermining the constitution. In any case, the criticism of the MT is to establish that these criticisms are not against you. (Music playing)"

In [2]:
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

NameError: name 'AutoModelForCausalLM' is not defined

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

RuntimeError: Failed to import transformers.models.mistral.modeling_mistral because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
/usr/local/lib/python3.10/dist-packages/transformer_engine_extensions.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

In [ ]:
messages = [
    {"role": "user", "content": "User I am giving you the title and short description of the news article in the format [Title:Description]. Give me relevant keywords like News, sports, entertainment,wellness or the related high level topics like important persons, political party, type of sports, etc. Please generate atleast five keyword's for the article \n {text}"},
]

In [ ]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

In [16]:
text

"Minister P. Rajiv's reaction to the criticism of M.T. Vastevana that there is no authoritarianism in Kerala is in the K.L.F.E.D. The minister criticized that the Centre is weakening the constitutional institutions. I don't think it's a question of whether or not we're going to be able to do it, but I think it's a question of whether or not we're going to be able to do it, and I think it's a question of whether or not we're going to be able to do it, because we're not going to be able to do it. In India, only one state, Kerala, has a parliamentary system that is democratic. This is the first time that such a reply or such a reference has been made by Minister P. Rajiv. In this article, we'll look at the history, culture, and ethics of the MTA, and then we'll take a closer look at the history and culture of the MTA, and then we'll look at the history of the MTA, and then we'll take a closer look at the history and ethics of the MTA. It's been a long time since I've been able to say that

Keywords: Minister P. Rajiv, Kerala, Constitutional Institutions, Central Government, Democracy, M.T. Vastevana, Politics, India. Additional keywords could include: Parliamentary System, Ethics, Culture, History, Constitution, Undermine, Criticism.

In [14]:
text2

'My book is coming out in about a week and people who are interested in that book will read it and they will realize that this paper Malayalam Manorama has manufactured a sentence and put it in quotes in the headline which is not there I think a responsible publication like Malayalam Manorama when it is putting a written manuscript in quotes at least it should be true to what is written my part Never, ever, not a single sentence in that book accuses my party, insulting my party and my work for 55 years. It\'s not fair for the author to make a headline and put it in court. From 1985 to 1985 when I was in Delhi and I was called Rita because of the emergency and it\'s about my experience in Delhi in the trade unions in collectively trying to build a women\'s organization and my and in that the struggle of a woman to establish in that book the struggle of a young woman to establish an independent identity in politics is part of my own experience but nowhere in this book have I ever made th

Keywords: Brinda Karat, Book, Headline, Malayalam Manorama, Politics, Communists, CPI-M, Memoir, Freedom of Speech, Experience, Independence, Party, Journalism, Ethics. Additional keywords could include: Misrepresentation, Accuracy, Responsibility, Public Document, Context, Young Women, Patriarchy, Indian Politics.